# --- Day 5: Sunny with a Chance of Asteroids ---

## ---Part One---

You're starting to sweat as the ship makes its way toward Mercury. The Elves suggest that you get the air conditioner working by upgrading your ship computer to support the Thermal Environment Supervision Terminal.

The Thermal Environment Supervision Terminal (TEST) starts by running a diagnostic program (your puzzle input). *The TEST diagnostic program will run on your existing Intcode computer after a few modifications:*

***First, you'll need to add two new instructions:***

* Opcode 3 takes a single integer as input and saves it to the position given by its only parameter:
> * For example, the instruction 3,50 would take an input value and store it at address 50.
* Opcode 4 outputs the value of its only parameter.
> * For example, the instruction 4,50 would output the value at address 50.

Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

**Second, you'll need to add support for parameter modes:**

* Each parameter of an instruction is handled based on its parameter mode. 
>Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory. <br> Until now, all parameters have been in position mode.

* Now, your ship computer will also need to handle parameters in mode 1, immediate mode.
>In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50.

**Parameter modes are stored in the same value as the instruction's opcode.**
* The opcode is a two-digit number based only on the ones and tens digit of the value
> * That is, the opcode is the rightmost two digits of the first value in an instruction
* Parameter modes are single digits, one per parameter, read right-to-left from the opcode: 
> * The first parameter's mode is in the hundreds digit
> * The second parameter's mode is in the thousands digit 
> * The third parameter's mode is in the ten-thousands digit, etc. 
> * Any missing modes are 0.

**For example, consider the program `1002,4,3,4,33`.**

> The first instruction, `1002,4,3,4`, is a multiply instruction:
>* The rightmost two digits of the first value, 02, indicate opcode 2, multiplication
>* Then, going right to left, the parameter modes are:
>* 0 (hundreds digit)
>* 1 (thousands digit)
>* and 0 (ten-thousands digit, not present and therefore zero):

```
ABCDE
 1002

DE - two-digit opcode,      02 == opcode 2
 C - mode of 1st parameter,  0 == position mode
 B - mode of 2nd parameter,  1 == immediate mode
 A - mode of 3rd parameter,  0 == position mode,
                                  omitted due to being a leading zero
```

                                  
**This instruction multiplies its first two parameters:**
>* The first parameter, `4` in position mode, works like it did before - its value is the value stored at address `4` (`33`). 
>* The second parameter, `3` in immediate mode, simply has value `3`. 
>* The result of this operation, `33 * 3 = 99`, is written according to the third parameter, `4` in position mode, which also works like it did before
>* `99` is written to address `4`.

**Parameters that an instruction writes to will never be in immediate mode.** 

**Finally, some notes:**

* It is important to remember that the instruction pointer should increase by the number of values in the instruction after the instruction finishes. Because of the new instructions, this amount is no longer always `4`.
* Integers can be negative: `1101,100,-1,4,0` is a valid program (find `100 + -1`, store the result in position `4`).

**The TEST diagnostic program will start by requesting from the user the ID of the system to test by running an input instruction:**<br>
* **Provide it 1**, the ID for the ship's air conditioner unit.
* It will then perform a series of diagnostic tests confirming that various parts of the Intcode computer, like parameter modes, function correctly. 
> * For each test, it will run an output instruction indicating how far the result of the test was from the expected value, where 0 means the test was successful. 
> * Non-zero outputs mean that a function is not working correctly; check the instructions that were run before the output instruction to see which one failed.

* Finally, the program will output a diagnostic code and immediately halt. 
> * This final output isn't an error; an output followed immediately by a halt means the program finished. If all outputs were zero except the diagnostic code, the diagnostic program ran successfully.

***After providing 1 to the only input instruction and passing all the tests, what diagnostic code does the program produce?***

In [1]:
def get_instructions(instruction, pointer):
    '''
    Takes the instruction and its index location and returns the opcode, parameters and increments the index by 1
    '''
    pointer = pointer + 1
    return instruction % 100 , instruction // 100, pointer

In [2]:
def mode_0(program, value):
    '''
    Mode 0 (position mode):
        The parameter to be interpreted as a position
    EX:
        If the parameter is 50, its value is the value stored at address 50
    '''
    return program[value]

def mode_1(program, value):
    '''
    Mode 1 (immediate mode):
        The parameter is interpreted as a value
    EX:
        If the parameter is 50, its value is simply 50'''
    return value

mode_dict = {0 : mode_0, 1 : mode_1}

In [3]:
def opcode_1(program, modes, pointer):
    '''
    Adds numbers from two positions and stores them in a third position
    '''
    modes = [ (modes // 10**n % 10) for n in range(3) ]
    parameters = [ mode_dict[modes[index-pointer]](program, program[index]) for index in range(pointer, pointer+2) ]
    program[mode_0(program, pointer+2)] = parameters[0] + parameters[1]
    pointer = pointer + 3
    return program, pointer

def opcode_2(program, modes, pointer):
    '''
    Multiplies numbers from two positions and stores them in a third position
    '''
    modes = [ (modes // 10**n % 10) for n in range(3) ]
    parameters = [ mode_dict[modes[index-pointer]](program, program[index]) for index in range(pointer, pointer+2) ]
    program[mode_0(program, pointer+2)] = parameters[0] * parameters[1]
    pointer = pointer + 3
    return program, pointer

def opcode_3(program, input_int, pointer):
    '''
    Takes a single integer as input and saves it to the position given by its only parameter
    '''
    program[program[pointer]] = input_int
    pointer = pointer + 1
    return program, pointer

def opcode_4(program, parameter, pointer):
    '''
    Outputs the value of its only parameter
    '''
    out = mode_dict[parameter](program,program[pointer])
    print( f"Output: {out}" )
    
    pointer = pointer + 1
        
    return program, pointer

def opcode_99(program, parameters=0, pointer=0):
    '''
    Halts the program.
    '''
    print("Optcode 99 : Program has halted.")
    pointer = len(program) + 1
    return program, pointer

op_dict = {1 : opcode_1, 2 : opcode_2, 3 : opcode_3, 4 : opcode_4, 99 : opcode_99}

In [4]:
def run_program(program, input_int=1, pointer=0):
    # Knowing that only the first opcode requires input we run this first
    opcode, modes, pointer = get_instructions(program[pointer], pointer)
    program, pointer = op_dict[opcode](program, input_int, pointer)
    
    while pointer < len(program):
        opcode, modes, pointer = get_instructions(program[pointer], pointer)
        program, pointer = op_dict[opcode](program, modes, pointer)

### Examples for testing modes

In [5]:
example1 = [1002, 4, 3, 4, 33]
opcode_1(example1,10,1)

([1002, 4, 3, 4, 36], 4)

In [6]:
example2 = [1002, 4, 3, 4, 33]
opcode_2(example2,10,1)

([1002, 4, 3, 4, 99], 4)

In [7]:
example3 = [1001,8,3,9,1002,8,3,8,33,0]
pointer = 0
while pointer < len(example3):
    opcode, modes, pointer = get_instructions(example3[pointer], pointer)
    example3, pointer = op_dict[opcode](example3, modes, pointer)
example3

Optcode 99 : Program has halted.


[1001, 8, 3, 9, 1002, 8, 3, 8, 99, 36]

### Lets see if it works for the day 5 input:

In [8]:
with open('day5.txt') as f:
    intcode = [int(i) for i in f.read().split(',')]
    run_program(intcode)

Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 12428642
Optcode 99 : Program has halted.


## --- Part Two ---

The air conditioner comes online! Its cold air feels good for a while, but then the TEST alarms start to go off. Since the air conditioner can't vent its heat anywhere but back into the spacecraft, it's actually making the air inside the ship warmer.

Instead, you'll need to use the TEST to extend the thermal radiators. Fortunately, the diagnostic program (your puzzle input) is already equipped for this. Unfortunately, your Intcode computer is not.

**Your computer is only missing a few opcodes:**

* Opcode 5 is jump-if-true: 
>* If the first parameter is non-zero
>* Set the instruction pointer to the value from the second parameter. 
>* Otherwise, do nothing.

* Opcode 6 is jump-if-false:
>* If the first parameter is zero
>* Set the instruction pointer to the value from the second parameter
>* Otherwise, do nothing.

* Opcode 7 is less than: 
>* If the first parameter is less than the second parameter
>* Store `1` in the position given by the third parameter
>* Otherwise, store `0`.


* Opcode 8 is equals:
>* If the first parameter is equal to the second parameter
>* Store `1` in the position given by the third parameter
>* Otherwise, store `0`.

**Like all instructions, these instructions need to support parameter modes as described above.**

Normally, after an instruction is finished, the instruction pointer increases by the number of values in that instruction. However, ***if the instruction modifies the instruction pointer, that value is used and the instruction pointer is not automatically increased.***

**The repititive lines of code in all the opcode's is exactly why I think it'll be much better to make a class**

In [9]:
def opcode_5(program, modes, pointer):
    '''
    Jump-if-true:
    If first parameter is non-zero; set instruction pointer to second parameter
    Else do nothing
    '''
    modes = [ (modes // 10**n % 10) for n in range(2) ]
    parameters = [mode_dict[modes[index-pointer]](program, program[index]) for index in range(pointer, pointer+2)]
    if parameters[0] != 0:
        pointer = parameters[1]
    else:
        pointer = pointer + 2

    return program, pointer

def opcode_6(program, modes, pointer):
    '''
    Jump-if-false:
    If the first parameter is zero; set instruction pointer to second parameter
    Else do nothing
    '''
    modes = [ (modes // 10**n % 10) for n in range(2) ]
    parameters = [mode_dict[modes[index-pointer]](program, program[index]) for index in range(pointer, pointer+2)]
    
    if parameters[0] == 0:
        pointer = parameters[1]
    else:
        pointer = pointer + 2
        
    return program, pointer

def opcode_7(program, modes, pointer):
    '''
    Less than:
    If first parameter is less than second parameter
    Store 1 in position given by third parameter
    Else store 0
    '''
    modes = [ (modes // 10**n % 10) for n in range(3) ]
    parameters = [ mode_dict[modes[index-pointer]](program, program[index]) for index in range(pointer, pointer+2) ]
    if parameters[0] < parameters[1]:
        program[mode_0(program, pointer+2)] = 1
    else:
        program[mode_0(program, pointer+2)] = 0
    
    pointer = pointer + 3
    
    return program, pointer

def opcode_8(program, modes, pointer):
    '''
    Equals:
    If first parameter is equal to the second parameter
    Store 1 in position given by third parameter
    Else store 0
    '''
    modes = [ (modes // 10**n % 10) for n in range(3) ]
    parameters = [ mode_dict[modes[index-pointer]](program, program[index]) for index in range(pointer, pointer+2)]
    if parameters[0] == parameters[1]:
        program[mode_0(program, pointer+2)] = 1
    else:
        program[mode_0(program, pointer+2)] = 0
    
    pointer = pointer + 3
    
    return program, pointer

op_dict[5] = opcode_5
op_dict[6] = opcode_6
op_dict[7] = opcode_7
op_dict[8] = opcode_8

**For example, here are several programs that take one input, compare it to the value 8, and then produce one output:**

> `3,9,8,9,10,9,4,9,99,-1,8` - Using position mode, consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
> `3,9,7,9,10,9,4,9,99,-1,8` - Using position mode, consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
> `3,3,1108,-1,8,3,4,3,99` - Using immediate mode, consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
> `3,3,1107,-1,8,3,4,3,99` - Using immediate mode, consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).

In [10]:
# Checking output equal to 8 position mode
equal_high = [3,9,8,9,10,9,4,9,99,-1,8]
run_program(equal_high,1)
equal_equal = [3,9,8,9,10,9,4,9,99,-1,8]
run_program(equal_equal,8)
equal_low = [3,9,8,9,10,9,4,9,99,-1,8]
run_program(equal_low,-8)

Output: 0
Optcode 99 : Program has halted.
Output: 1
Optcode 99 : Program has halted.
Output: 0
Optcode 99 : Program has halted.


In [11]:
# Checking output less than 8 position mode
less_false = [3,9,7,9,10,9,4,9,99,-1,8]
run_program(less_false,10) 
less_true =[3,9,7,9,10,9,4,9,99,-1,8]
run_program(less_false,-7)
less_equal = [3,9,7,9,10,9,4,9,99,-1,8]
run_program(less_equal,8)

Output: 0
Optcode 99 : Program has halted.
Output: 1
Optcode 99 : Program has halted.
Output: 0
Optcode 99 : Program has halted.


In [12]:
# Checking output equal to 8 immediate mode
imm_high = [3,3,1108,-1,8,3,4,3,99]
run_program(imm_high,10)
imm_equal = [3,3,1108,-1,8,3,4,3,99]
run_program(imm_equal,8)
imm_neg = [3,3,1108,-1,8,3,4,3,99]
run_program(imm_neg,-8)

Output: 0
Optcode 99 : Program has halted.
Output: 1
Optcode 99 : Program has halted.
Output: 0
Optcode 99 : Program has halted.


In [13]:
# Checking output less than 8 immediate mode
imm_less = [3,3,1107,-1,8,3,4,3,99]
run_program(imm_less,-10)
imm_more = [3,3,1107,-1,8,3,4,3,99]
run_program(imm_more,8)
imm_same = [3,3,1107,-1,8,3,4,3,99]
run_program(imm_same,-8)

Output: 1
Optcode 99 : Program has halted.
Output: 0
Optcode 99 : Program has halted.
Output: 1
Optcode 99 : Program has halted.


**Here are some jump tests that take an input, then output 0 if the input was zero or 1 if the input was non-zero:**

> `3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9` (using position mode)<br>
> `3,3,1105,-1,9,1101,0,0,12,4,12,99,1` (using immediate mode)

In [14]:
pos_jump_one = [3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]
run_program(pos_jump_one,1)
pos_jump_zero = [3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]
run_program(pos_jump_zero,0)
pos_jump_neg = [3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]
run_program(pos_jump_neg,-1)

Output: 1
Optcode 99 : Program has halted.
Output: 0
Optcode 99 : Program has halted.
Output: 1
Optcode 99 : Program has halted.


In [15]:
imm_jump_one = [3,3,1105,-1,9,1101,0,0,12,4,12,99,1]
run_program(imm_jump_one,1)
imm_jump_zero = [3,3,1105,-1,9,1101,0,0,12,4,12,99,1]
run_program(imm_jump_zero,0)
imm_jump_neg = [3,3,1105,-1,9,1101,0,0,12,4,12,99,1]
run_program(imm_jump_neg,-1)

Output: 1
Optcode 99 : Program has halted.
Output: 0
Optcode 99 : Program has halted.
Output: 1
Optcode 99 : Program has halted.


**Here's a larger example:**

> `3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,`<br>
> `1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,`<br>
> `999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99`

The above example program uses an input instruction to ask for a single number. The program will then output 999 if the input value is below 8, output 1000 if the input value is equal to 8, or output 1001 if the input value is greater than 8.

In [16]:
large_high = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
run_program(large_high, 10)
large_low = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
run_program(large_low, 5)
large_equal = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
run_program(large_equal, 8)

Output: 1001
Optcode 99 : Program has halted.
Output: 999
Optcode 99 : Program has halted.
Output: 1000
Optcode 99 : Program has halted.


This time, when the TEST diagnostic program runs its input instruction to get the ID of the system to test, provide it 5, the ID for the ship's thermal radiator controller. 
This diagnostic test suite only outputs one number, the diagnostic code.

**What is the diagnostic code for system ID 5?**

In [17]:
with open('day5.txt') as f:
    intcode = [ int(i) for i in f.read().split(',')]
    run_program(intcode, 5)

Output: 918655
Optcode 99 : Program has halted.
